# SNS Sentiment Analysis BERT

Analyze social media texts and measure potential inflammatory / offensive language with a BERT neural network.

(This does not work, unable to pass data through the BERT model, Iter nor Tensor conversions are working at all for the data and model).

In [ ]:
import ast
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from transformers import TFBertModel, BertTokenizer

In [3]:
# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased', output_hidden_states=False, return_dict=False)
bert_model.trainable = False

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [5]:
print("Tensorflow Version : ", tf.__version__)
print("Transformers Version : ", transformers.__version__)

Tensorflow Version :  2.14.0
Transformers Version :  4.35.0


# Load the data

In [12]:
# Load dataset
train = pd.read_csv("./datasets/processed/train.csv")
X_train = train[train.columns[0]].apply(ast.literal_eval)[:800]
y_train = train[train.columns[1]].astype(int)[:200]
print(X_train.shape, y_train.shape)

test = pd.read_csv("./datasets/processed/test.csv")
X_test = test[test.columns[0]].apply(ast.literal_eval)[:800]
y_test = test[test.columns[1]].astype(int)[:200]
print(X_test.shape, y_test.shape)

(800,) (200,)
(800,) (200,)


In [13]:
train_text = [" ".join(s) for s in X_train]
test_text = [" ".join(s) for s in X_test]

In [7]:
example_input = tokenizer(["Test sentence", "This is a test sentence"], padding='max_length', truncation=True, return_tensors='tf')
example_input

{'input_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[ 101, 3231, 6251, ...,    0,    0,    0],
       [ 101, 2023, 2003, ...,    0,    0,    0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [8]:
BATCH_SIZE = 4
MAX_LENGTH = 128

def data_generator(text, labels, batch_size: int) -> (dict, tf.Tensor):
    num_samples = len(text)
    i = 0
    while True:
        x_data, y_data = [], []
        for j in range(batch_size):
            start = i * batch_size
            try:
                x_data.append(text[start + j])
                y_data.append(labels[start + j])
            except KeyError:
                print("INDEX:", j, "START:", start, "\n", labels)
            i += 1


        if i * batch_size >= num_samples:
            i = 0

        batch_x = tokenizer(x_data, return_tensors="tf", max_length=MAX_LENGTH, padding='max_length', truncation=True)
        batch_y = tf.convert_to_tensor(y_data)

        yield dict(batch_x), batch_y

In [14]:
train_data_gen = data_generator(train_text, y_train, BATCH_SIZE)
test_data_gen = data_generator(test_text, y_test, BATCH_SIZE)

# train_text_t = tokenizer(train_text, return_tensors="tf", max_length=MAX_LENGTH, padding='max_length', truncation=True)
# test_text_t = tokenizer(test_text, return_tensors="tf", max_length=MAX_LENGTH, padding='max_length', truncation=True)

# train_dataset = tf.data.Dataset.from_tensor_slices((train_text_t['input_ids'], train_text_t['attention_mask'], y_train))
# test_dataset = tf.data.Dataset.from_tensor_slices((test_text_t['input_ids'], test_text_t['attention_mask'], y_test))
# 
# train_dataset = train_dataset.batch(BATCH_SIZE)
# test_dataset = test_dataset.batch(BATCH_SIZE)

In [15]:
def create_bert_model():
    input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='attention_mask')
    token_type_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='token_type_ids')

    bert_output = bert_model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    sequence_output = bert_output[0]
    # Take the [CLS] token representation
    pooled_output = sequence_output[:, 0, :]

    x = tf.keras.layers.Dropout(0.2)(pooled_output)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[input_ids, attention_mask, token_type_ids], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=3e-5), loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [10]:
class SNSBERTClassification(tf.keras.Model):
    def __init__(self, bert_model_, num_classes=1):
        super().__init__()
        self.bert = bert_model_
        self.fc = tf.keras.layers.Dense(num_classes, activation="sigmoid")

    def call(self, inputs, training=None, mask=None):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [11]:
bert_classifier = SNSBERTClassification(bert_model, num_classes=1)

bert_classifier.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
b = create_bert_model()

X_train_tokens = tokenizer.batch_encode_plus(
    train_text[:200],
    max_length=MAX_LENGTH,
    padding='max_length',
    truncation=True,
    return_tensors='tf'
)

# Tokenize the testing data
X_test_tokens = tokenizer.batch_encode_plus(
    test_text[:200],
    max_length=MAX_LENGTH,
    padding='max_length',
    truncation=True,
    return_tensors='tf'
)

In [ ]:
b.fit(
    [X_train_tokens['input_ids'], X_train_tokens['attention_mask'], X_train_tokens['token_type_ids']],
    y_train,
    validation_data=([X_test_tokens['input_ids'], X_test_tokens['attention_mask'], X_test_tokens['token_type_ids']], y_test),
    epochs=3,
    batch_size=32
)

In [ ]:
bert_history = bert_classifier.fit(
    train_data_gen,
    epochs=3,
    batch_size=BATCH_SIZE,
    steps_per_epoch=len(train_text) // BATCH_SIZE,
    validation_data=test_data_gen,
    validation_steps=BATCH_SIZE * 4,
    validation_batch_size=BATCH_SIZE
)

In [ ]:
model_ = tf.keras.Sequential([
    bert_model,
    tf.keras.layers.Lambda(lambda x: x[0][:,0,:]),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model_.fit(
    train_data_gen,
    batch_size=BATCH_SIZE,
    steps_per_epoch=len(train_text) // BATCH_SIZE,
    validation_data=test_data_gen,
    validation_steps=len(test_text) // BATCH_SIZE,
    validation_batch_size=BATCH_SIZE,
    epochs=3
)

In [ ]:
bert_classifier.evaluate_generator(test_data_gen)